## General Model Evaluation Examples

In [ ]:
import pickle
def readPicklefromTraining(fname):
    filename = "Model_Training/pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [ ]:
# import the model predictions that you want to evaluate
pos_res = readPicklefromTraining("predictions_POS_33") # one POS model
pho_res = readPicklefromTraining("predictions_PHO1") # one Phoneme Model
rid_res = readPicklefromTraining("predictions_RID_33") # one RID model
dummy_res = readPicklefromTraining("predictions_dummy_33") # and the dummy model

# import the test labels for comparison
test_labels = readPickle("cnn_data_inputs/test_POS_labels")

In [ ]:
# for each of the predicted labels in the model predictions, make a comparison with the actual test label counterpart 
import numpy as np
PHO_correct_prediction_indices = list()
test_label_list = test_labels
counter = 0
PHO_predict_array = []
for array, label in zip(pho_res, test_labels):
    predict = np.argmax(array, axis=None)
    if predict == label: # the prediction matches the true value, add 1 to the prediction array for that model
        PHO_correct_prediction_indices.append(test_label_list.index(label))
        counter += 1
        PHO_predict_array.append(1) 
    else: # if not, add a 0
        PHO_predict_array.append(0)
        
print(counter)
print("Total percentage of accurate labels in the test set is", "{0:.2%}".format(counter/len(test_labels)), "for the PHO model")

POS_correct_prediction_indices = list()
test_label_list = test_labels
counter = 0
POS_predict_array = []
for array, label in zip(pos_res, test_labels):
    predict = np.argmax(array, axis=None)
    if predict == label: # the prediction matches the true value, add 1 to the prediction array for that model
        POS_correct_prediction_indices.append(test_label_list.index(label))
        counter += 1
        POS_predict_array.append(1)
    else: # if not, add a 0
        POS_predict_array.append(0)
print(counter)
print("Total percentage of accurate labels in the test set is", "{0:.2%}".format(counter/len(test_labels)), "for the POS model")

RID_correct_prediction_indices = list()
test_label_list = test_labels
counter = 0
RID_predict_array = []
for array, label in zip(rid_res, test_labels):
    predict = np.argmax(array, axis=None)
    if predict == label: # the prediction matches the true value, add 1 to the prediction array for that model
        RID_correct_prediction_indices.append(test_label_list.index(label))
        counter += 1
        RID_predict_array.append(1)
    else: # if not, add a 0
        RID_predict_array.append(0)
        
print(counter)
print("Total percentage of accurate labels in the test set is", "{0:.2%}".format(counter/len(test_labels)), "for the RID model")


dummy_correct_prediction_indices = list()
test_label_list = test_labels
counter = 0
dummy_predict_array = []
for array, label in zip(dummy_res, test_labels):
    predict = np.argmax(array, axis=None)
    if predict == label: # the prediction matches the true value, add 1 to the prediction array for that model
        dummy_correct_prediction_indices.append(test_label_list.index(label))
        counter += 1
        dummy_predict_array.append(1)
    else: # if not, add a 0
        dummy_predict_array.append(0)
        
print(counter)
print("Total percentage of accurate labels in the test set is", "{0:.2%}".format(counter/len(test_labels)), "for the dummy model")

429 correct labels<br>
Total percentage of accurate labels in the test set is 9.41% for the PHO model<br>
1383 correct labels<br>
Total percentage of accurate labels in the test set is 30.33% for the POS model <br>
1345 correct labels<br>
Total percentage of accurate labels in the test set is 29.50% for the RID model<br>
1340 correct labels<br>
Total percentage of accurate labels in the test set is 29.39% for the dummy model<br>
### By changing the predictions in the second cell, we can evaluate the scores of each individiual model. These predictions are used as examples here. 


### By using the following script, we can also add different models and see how their combinations work

In [ ]:
combined_predict_array = []
# for an example, combine the given Pho, POS and RID models
for pho, pos, rid in zip(PHO_predict_array, POS_predict_array, RID_predict_array):
    if pho == 1 or pos == 1 or rid == 1: # when any of the predictions are correct, count as one
        combined_predict_array.append(1)
    else:
        combined_predict_array.append(0)

In [ ]:
print("Total percentage of accurate labels in the test set is", "{0:.2%}".format(combined_predict_array.count(1)/len(test_labels)), "for the combined predictions")


Total percentage of accurate labels in the test set is 32.08% for the combined predictions

## Genre Predictions, Confusion Matrices, Training Loss/Accuracy Plots

In [ ]:
from keras.utils.vis_utils import plot_model
from keras.models import load_model


In [3]:
import pickle
def writePickle( Variable, fname):
    filename = fname +".pkl"
    f = open("pickle_vars/"+filename, 'wb')
    pickle.dump(Variable, f, protocol=4)
    f.close()
def readPickle(fname):
    filename = "pickle_vars/"+fname +".pkl" # notice the ../ addition to reach out to variables from the parent directory
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj
def readHistory(fname):
    filename = "Model_Training/pickle_vars/"+fname +".pkl" # notice the ../ addition to reach out to variables from the parent directory
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj
def readPicklefromTraining(fname):
    filename = "Model_Training/pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# use a mapping dictionary to map each genre to one of the desired genre classes
genre_mapping = {"Folk Rock" : "Rock", "Soft Rock" : "Rock", "Britpop": "Pop", "Black Metal": "Metal", \
                 "Classic Rock": "Rock",  'Dance-Pop' : 'Pop', 'Electro': 'Electronic', "Post-Rock": "Rock", \
                "Power Pop" : "Pop", "Rock 'N' Roll" : "Rock", 'Psychedelic Rock': "Rock",'Thrash Metal':"Metal",\
                 'American Folk': 'Folk', 'Dream Pop': 'Pop', 'Art Rock':'Rock', 'Reggae': 'Reggae',\
                 'Indie Folk': 'Folk', 'Jazz': "Jazz & Blues", 'Death Metal': "Metal", 'Progressive Rock': 'Rock',\
                 'Pop Rock': 'Pop', 'Electronic':'Electronic', 'Hard Rock':'Rock', 'R&B': 'R&B', 'Indie Pop':'Pop',\
                 'Heavy Metal':'Metal', 'Blues': "Jazz & Blues", 'Folk': "Folk", 'Country': 'Country', \
                 'Indie Rock':'Rock', 'Hip Hop': 'Hip Hop & Rap', 'Alternative Rock':'Rock', 'Rock':'Rock', 'Pop':'Pop' }

id2Artists = readPickle("indexing/id2Artist")
Artists2genres = readPickle("artist_to_genre")
test_labels = readPickle("cnn_data_inputs/test_POS_labels")

In [ ]:
# use a function for genre mapping
genre_test_labels = list()
genre_test_labels_MAPPED = list()
for ids in test_labels:
    genre_test_labels.append(Artists2genres[id2Artists[ids]])
    genre_test_labels_MAPPED.append(genre_mapping[Artists2genres[id2Artists[ids]]])

def genre_checker(predictions):
    all_genre_predictions = list()
    all_MAPPED_genre_predictions = list()
    for array in predictions: # here we need to write whatever prediction list we need to analyze
        predict = np.argmax(array, axis=None)
        all_genre_predictions.append(Artists2genres[id2Artists[predict]])
        all_MAPPED_genre_predictions.append(genre_mapping[Artists2genres[id2Artists[predict]]])

    total_count = 0
    total_mapped_count = 0
    correct_count = 0
    correct_mapped_count = 0
    for real_genre, predicted_genre in zip(genre_test_labels, all_genre_predictions):
        total_count += 1
        if real_genre == predicted_genre:
            correct_count +=1
    for real_genre, predicted_mapped_genre in zip(genre_test_labels_MAPPED, all_MAPPED_genre_predictions):
        total_mapped_count += 1
        if real_genre == predicted_mapped_genre:
            correct_mapped_count += 1
    print("Unmapped genre prediction accuracy for this prediction set is", correct_count/total_count, \
          "\n while the mapped genre prediction accuracy for the same prediction set is", correct_mapped_count/total_mapped_count)

    
    

In [ ]:
# use another function for genre confusion matrix
genre_test_labels_set = list(set(genre_test_labels_MAPPED))
genre_indexing = {v: k for v, k in enumerate(genre_test_labels_set)}

def confusion_matrix(predictions):
    
    for ids in test_labels:
        genre_test_labels_MAPPED.append(genre_mapping[Artists2genres[id2Artists[ids]]])
    all_MAPPED_genre_predictions = list()
    for array in predictions: # here we need to write whatever prediction list we need to analyze
        predict = np.argmax(array, axis=None)
        all_MAPPED_genre_predictions.append(genre_mapping[Artists2genres[id2Artists[predict]]])
    
    confusion_dict = dict()
    for genre in genre_test_labels_set:
        inter_dict = dict()
        for item in genre_test_labels_set:
            inter_dict[item] = 0
        confusion_dict[genre] = inter_dict
    for actual, predicted in zip(genre_test_labels_MAPPED, all_MAPPED_genre_predictions):
        confusion_dict[actual][predicted] += 1
    
    cfm = pd.DataFrame(columns=genre_test_labels_set)
    for genre in genre_test_labels_set:
        cfm = cfm.append(confusion_dict[genre], ignore_index=True)
    cfm_new = cfm.rename(genre_indexing)
    
    f1_score_dict = dict()
    for genre in confusion_dict.keys():
        TP = confusion_dict[genre][genre]
        total_actuals = 0
        for value in confusion_dict[genre].values():
            total_actuals += value
        FN = total_actuals - TP
        total_predictions = 0
        everything = 0
        for genre2 in confusion_dict.keys():
            total_predictions += confusion_dict[genre2][genre]
            everything += sum(list(confusion_dict[genre2].values()))
        FP = total_predictions - TP
        TN = everything - (TP+FP+FN)
        try:
            Precision = TP/(TP+FP)
        except:
            Precision = 0
        Recall = TP/(TP+FN)
        print(genre, Precision, Recall)
        try:
            f1_score_dict[genre] = (2*Precision*Recall)/(Precision+Recall)
        except:
            f1_score_dict[genre] = 0.01
    average_f1_score = sum(list(f1_score_dict.values()))/10
    

    #confusion_matrix_perc.loc['Column_Total']= confusion_matrix_perc.sum(numeric_only=True, axis=0)
    #confusion_matrix_perc.loc[:,'Row_Total'] = confusion_matrix_perc.sum(numeric_only=True, axis=1)
    return cfm_new, f1_score_dict, average_f1_score


### For the best POS model that overfits (but not the worst overfitting model)

In [ ]:
test_labels = readPickle("cnn_data_inputs/test_POS_labels")


In [ ]:
history = readHistory("history_POS_25ep_128bch_05Drop_33filter_batchnorm")
model_predictions = readPicklefromTraining("predictions_POS_25ep_128bch_05Drop_33filter_batchnorm")


In [ ]:
import numpy as np
POS_correct_prediction_indices = list()
test_label_list = test_labels
counter = 0
POS_predict_array = []
for array, label in zip(model_predictions, test_labels):
    predict = np.argmax(array, axis=None)
    if predict == label:
        POS_correct_prediction_indices.append(test_label_list.index(label))
        counter += 1
        POS_predict_array.append(1)
    else:
        POS_predict_array.append(0)
        
print(counter)
print("Total percentage of accurate labels in the test set is", "{0:.2%}".format(counter/len(test_labels)), "for the PHO model")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True)

fig.suptitle('POS ONLY MODEL WITH NO SIZE RESTRICTIONS')

    
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
ax1.plot(epochs, loss, color='red', label='Training loss')
ax1.plot(epochs, val_loss, color='green', label='Validation loss')
ax1.set(ylabel="Loss")
ax1.legend()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
ax2.plot(epochs, acc, color='red', label='Training acc')
ax2.plot(epochs, val_acc, color='green', label='Validation acc')
ax2.set(xlabel = "Number of Epochs", ylabel="Accuracy")
ax2.legend()

plt.show()

In [ ]:
genre_checker(model_predictions)

In [ ]:
confusion_matrix(model_predictions)[0]

In [ ]:
confusion_matrix(model_predictions)[1]

In [ ]:
print("The average f1 score for this genre prediction model is:", confusion_matrix(model_predictions)[2])

### The 10_10 (Reduced) Pho Model

In [ ]:
history = readHistory("history_PHO_10_10_25ep_64bch_05Drop_33filter_batchnorm")
model_predictions = readPicklefromTraining("predictions_PHO_10_10_batchnorm_25ep_64bch")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True)

fig.suptitle('PHO ONLY MODEL WITH (10,10) SIZE RESTRICTIONS')

    
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
ax1.plot(epochs, loss, color='red', label='Training loss')
ax1.plot(epochs, val_loss, color='green', label='Validation loss')
ax1.set(ylabel="Loss")
ax1.legend()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
ax2.plot(epochs, acc, color='red', label='Training acc')
ax2.plot(epochs, val_acc, color='green', label='Validation acc')
ax2.set(xlabel = "Number of Epochs", ylabel="Accuracy")
ax2.legend()

plt.show()

In [ ]:
genre_checker(model_predictions)

In [ ]:
confusion_matrix(model_predictions)[0]

In [ ]:
confusion_matrix(model_predictions)[1]

In [ ]:
print("The average f1 score for this genre prediction model is:", confusion_matrix(model_predictions)[2])

### RID_Reduced (10,3) Model 

In [ ]:
history = readHistory("history_RID_reduced_25ep_64bch_05Drop_11filter_batchnorm")
model_predictions = readPicklefromTraining("predictions_RID_reduced_batchnorm_25ep_64bch")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True)

fig.suptitle('RID ONLY MODEL WITH (10,3) SIZE RESTRICTIONS')

    
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
ax1.plot(epochs, loss, color='red', label='Training loss')
ax1.plot(epochs, val_loss, color='green', label='Validation loss')
ax1.set(ylabel="Loss")
ax1.legend()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
ax2.plot(epochs, acc, color='red', label='Training acc')
ax2.plot(epochs, val_acc, color='green', label='Validation acc')
ax2.set(xlabel = "Number of Epochs", ylabel="Accuracy")
ax2.legend()

plt.show()

In [ ]:
genre_checker(model_predictions)

In [ ]:
confusion_matrix(model_predictions)[0]

In [ ]:
confusion_matrix(model_predictions)[1]

In [ ]:
print("The average f1 score for this genre prediction model is:", confusion_matrix(model_predictions)[2])

### Direct genre predictions

Two different POS models (reduced and full respectively)

In [ ]:
model_predictions = readPicklefromTraining("predictions_POS_GENRE_reduced_batchnorm_25ep_64bch")
print(confusion_matrix(model_predictions)[1], confusion_matrix(model_predictions)[2])

In [ ]:
confusion_matrix(model_predictions)[0]

In [ ]:
model_predictions = readPicklefromTraining("predictions_POS_GENRE_25ep_128bch_05Drop_33filter_batchnorm")
print(confusion_matrix(model_predictions)[1], confusion_matrix(model_predictions)[2])

In [ ]:
confusion_matrix(model_predictions)[0]

Reduced PHO model

In [ ]:
model_predictions = readPicklefromTraining("predictions_PHO_GENRE_10_10_batchnorm_25ep_64bch")
print(confusion_matrix(model_predictions)[1], confusion_matrix(model_predictions)[2])

In [ ]:
confusion_matrix(model_predictions)[0]

Reduced RID model

In [ ]:
model_predictions = readPicklefromTraining("predictions_RID_GENRE_reduced_batchnorm_25ep_64bch")
print(confusion_matrix(model_predictions)[1], confusion_matrix(model_predictions)[2])

## Statistical Analysis for Certain Phoneme Groups

In [ ]:
ID_to_PHO = readPickle("ids2splitted_phonemes_dict")

In [ ]:
import matplotlib.pyplot as plt
from collections import OrderedDict

for song_id in list(ID_to_PHO.keys())[0:20]:
    song = ID_to_PHO[song_id]
    all_song_phonemes = list()
    for line in song:
        refined_line = [tag for tag in line if tag != ' ' and tag != 'PADDING']
        combined_line = ''.join(refined_line)
        if len(combined_line) >= 3:
            for i in range(len(combined_line)-3+1):
                all_song_phonemes.append(combined_line[i:i+3])

    all_phoneme_freq = defaultdict(int)
    for pho in all_song_phonemes:
        all_phoneme_freq[pho] += 1

    sorted_all_phoneme_freq = OrderedDict(sorted(all_phoneme_freq.items(), key=lambda v: v[1], reverse=True))
    refined_all_phoneme_freq = dict()
    for pho, count in sorted_all_phoneme_freq.items():
        if count > 1:
            refined_all_phoneme_freq[pho] = count
    refined_all_phoneme_freq = OrderedDict(sorted(refined_all_phoneme_freq.items(), key=lambda v: v[1], reverse=True))
    print("Total percentage of phonemes that occur at least 2 times in the song is", "{0:.2%}".format(len(refined_all_phoneme_freq)/len(sorted_all_phoneme_freq)))
    print("The number of phonemes that occur at least 2 times in the song is:", len(refined_all_phoneme_freq))
    
    plt.bar(refined_all_phoneme_freq.keys(), refined_all_phoneme_freq.values(), width=1, color='g')
    plt.show()

Since the songs generally consist of 3-gram phonemes that occur at least two times throughout the song, let's write a function that converts each song to phoneme n-grams, calculates their frequencies and assign values in terms of these frequencies 

In [ ]:
from collections import OrderedDict
from collections import defaultdict


def song_phoneme_freq(ngram): #ngram defines the number of phonetic signs that occur successively to form a basic rhytmical component
    counter = 0
    total_dataset_length = len(list(ID_to_PHO.keys()))
    ID_to_Full_Rhymes = dict() #this one is for all phonemes, but assigning 0.0 to the ones that occur seldom (i.e less than two times in the song)
    ID_to_Partial_Rhymes = dict() #this one is for phonemes that occur very frequently only
    for song_id, song in ID_to_PHO.items():
        counter += 1
        print(counter, "out of", total_dataset_length, "songs is being processed now")
        
        # the first step is to find all n-gram phonetical representations and their frequencies in a song
        all_song_phonemes = list()
        for line in song:
            refined_line = [tag for tag in line if tag != ' ' and tag != 'PADDING'] # first strip each line from blank spaces and paddings
            combined_line = ''.join(refined_line) # combine phonetical representations
            # create a list of n-gram phonetical representations 
            if len(combined_line) >= ngram:
                for i in range(len(combined_line)-ngram+1):
                    all_song_phonemes.append(combined_line[i:i+ngram])
        all_phoneme_freq = defaultdict(int) # this is for counting and recording the frequencies
        for pho in all_song_phonemes:
            all_phoneme_freq[pho] +=1
        sorted_all_phoneme_freq = OrderedDict(sorted(all_phoneme_freq.items(), key=lambda v: v[1], reverse=True)) # sort the frequencies in descending order
        refined_all_phoneme_freq = dict()
        for pho, count in sorted_all_phoneme_freq.items():
            if count > 1:
                refined_all_phoneme_freq[pho] = count
        refined_all_phoneme_freq = OrderedDict(sorted(refined_all_phoneme_freq.items(), key=lambda v: v[1], reverse=True))
        
        # for the top 9 most frequently occuring n-gram phonemes, assign values 0.95, 0.9, 0.85, .... , 0.55 respectively
        percentage_allocation = dict()
        perc = 1.0
        for phoneme in list(refined_all_phoneme_freq.keys())[0:9]:
            perc -= 0.05
            percentage_allocation[phoneme] = round(perc,1)
        # if there are more phoneme n-grams with less frequencies (which is the case 99% of the time)
        try: # assign 0.5 to the 10th most occuring phoneme n-gram frequency, and normalize the rest of the frequencies with respect to 0.5
            max_rest_freq = refined_all_phoneme_freq[list(refined_all_phoneme_freq.keys())[9]]
            for phoneme in list(refined_all_phoneme_freq.keys())[9:]:
                percentage_allocation[phoneme] = round(refined_all_phoneme_freq[phoneme]/max_rest_freq*0.5,1)
        except:
            print("There are at most 8 phonemes in song with id", song_id)
        
        # after mapping each n-gram phoneme to its respective normalized frequency value, 
        # we continue with converting the songs to these normalized values. 
        # one version assigns the value 0.0 also to the n-grams occuring less than 2 times, the other disregards them
        converted_song_rhymes_only = list() # <2 n-grams are disregarded
        converted_song_all_phonemes = list() # <2 n-grams are included and assigned 0.0
        for line in song:
            converted_line_rhymes_only = list()
            converted_line_all_phonemes = list()
            transformed_line = ''.join([tag for tag in line if tag != ' ' and tag != 'PADDING'])
            if len(transformed_line) >= ngram:
                for i in range(len(transformed_line)-ngram+1):
                    try:
                        converted_line_rhymes_only.append(percentage_allocation[transformed_line[i:i+ngram]])
                        converted_line_all_phonemes.append(percentage_allocation[transformed_line[i:i+ngram]])
                    except:
                        converted_line_all_phonemes.append(0.0)
            if len(converted_line_rhymes_only) > 0:
                converted_song_rhymes_only.append(converted_line_rhymes_only)
            if len(converted_line_all_phonemes) > 0:
                converted_song_all_phonemes.append(converted_line_all_phonemes)
        ID_to_Full_Rhymes[song_id] = converted_song_all_phonemes
        ID_to_Partial_Rhymes[song_id] = converted_song_rhymes_only

    return ID_to_Full_Rhymes, ID_to_Partial_Rhymes
    
        

In [ ]:
ID_to_Full_Rhymes, ID_to_Partial_Rhymes = song_phoneme_freq(3)

In [ ]:
writePickle(ID_to_Full_Rhymes, "ID_to_Full_Rhymes")
writePickle(ID_to_Partial_Rhymes, "ID_to_Partial_Rhymes")

Now it is time to find some statistics about the line lengths and songs lengths of these phoneme n-gram frequency versions of songs. According to the statistics, we'll apply certain resizing operations to substitute masking in CNNs 

In [ ]:
song_length_counts_full = list()
max_line_length_counts_full = list()
song_length_counts_partial = list()
max_line_length_counts_partial = list()
for song in ID_to_Full_Rhymes.values():
    song_length_counts_full.append(len(song))
    line_length_counts = list()
    for line in song:
        line_length_counts.append(len(line))
    max_line_length_counts_full.append(max(line_length_counts))
for song in ID_to_Partial_Rhymes.values():
    song_length_counts_partial.append(len(song))
    line_length_counts = list()
    for line in song:
        line_length_counts.append(len(line))
    max_line_length_counts_partial.append(max(line_length_counts))
    
from collections import OrderedDict

# for full rhymes, form ordered dictionaries that map song lengths to their frequency of occuring in the whole dataset
# also form another ordered dict for max line length frequencies
full_song_length_freq = defaultdict(int)
full_max_line_length_freq = defaultdict(int)
for length in song_length_counts_full:
    full_song_length_freq[length] += 1
sorted_full_song_length_freq = OrderedDict(sorted(full_song_length_freq.items(), key=lambda v: v[0], reverse=False))
for length in max_line_length_counts_full:
    full_max_line_length_freq[length] += 1
sorted_full_max_line_length_freq = OrderedDict(sorted(full_max_line_length_freq.items(), key=lambda v: v[0], reverse=False))
# do the same for partial rhymes
partial_song_length_freq = defaultdict(int)
partial_max_line_length_freq = defaultdict(int)
for length in song_length_counts_partial:
    partial_song_length_freq[length] += 1
sorted_partial_song_length_freq = OrderedDict(sorted(partial_song_length_freq.items(), key=lambda v: v[0], reverse=False))
for length in max_line_length_counts_partial:
    partial_max_line_length_freq[length] += 1
sorted_partial_max_line_length_freq = OrderedDict(sorted(partial_max_line_length_freq.items(), key=lambda v: v[0], reverse=False))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = sorted_partial_song_length_freq

fig, ax = plt.subplots(figsize=(20, 9))

x = range(len(data))
y = np.cumsum(list(data.values())).astype("float32")
y/=y.max()
y*=100.
#y = list(data.values())

ax.bar(x, y, label="Cumulative Song Lengths", fill=False)
ax.axhline(y=80.0, color = "r")
ax.axhline(y=60.0, color = "b")

plt.subplots_adjust(bottom=0.1)
plt.xticks(x, data.keys(), rotation='vertical')
ax.grid(True)
ax.legend(loc='center left')
ax.set_title('Song Length Frequency Distrubition for Partial N-Gram Phonemes')
ax.set_xlabel('Total Song Length in terms of Number of Lines')
ax.set_ylabel('Percentage of songs that have at least x lines')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = sorted_partial_max_line_length_freq

fig, ax = plt.subplots(figsize=(20, 9))

x = range(len(data))
y = np.cumsum(list(data.values())).astype("float32")
y/=y.max()
y*=100.
#y = list(data.values())

ax.bar(x, y, label="Cumulative Max Line Lengths per Song", fill=False)
ax.axhline(y=80.0, color = "r")
ax.axhline(y=60.0, color = "b")

plt.subplots_adjust(bottom=0.1)
plt.xticks(x, data.keys(), rotation='vertical')
ax.grid(True)
ax.legend(loc='center left')
ax.set_title('Max Line Length per Song Frequency Distrubition for Partial N-Gram Phonemes')
ax.set_xlabel('Total Max Line Length in terms of Number of Ngrams')
ax.set_ylabel('Percentage of songs that have a line with at most x ngrams')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = sorted_partial_song_length_freq

fig, ax = plt.subplots(figsize=(20, 9))

x = range(len(data))
y = np.cumsum(list(data.values())).astype("float32")
y/=y.max()
y*=100.
#y = list(data.values())

ax.bar(x, y, label="Cumulative Song Lengths", fill=False)
ax.axhline(y=80.0, color = "r")
ax.axhline(y=60.0, color = "b")

plt.subplots_adjust(bottom=0.1)
plt.xticks(x, data.keys(), rotation='vertical')
ax.grid(True)
ax.legend(loc='center left')
ax.set_title('Song Length Frequency Distrubition for Full N-Gram Phonemes')
ax.set_xlabel('Total Song Length in terms of Number of Lines')
ax.set_ylabel('Percentage of songs that have at least x lines')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = sorted_full_max_line_length_freq

fig, ax = plt.subplots(figsize=(20, 9))

x = range(len(data))
y = np.cumsum(list(data.values())).astype("float32")
y/=y.max()
y*=100.
#y = list(data.values())

ax.bar(x, y, label="Cumulative Max Line Lengths per Song", fill=False)
ax.axhline(y=80.0, color = "r")
ax.axhline(y=60.0, color = "b")

plt.subplots_adjust(bottom=0.1)
plt.xticks(x, data.keys(), rotation='vertical')
ax.grid(True)
ax.legend(loc='center left')
ax.set_title('Max Line Length per Song Frequency Distrubition for Full N-Gram Phonemes')
ax.set_xlabel('Total Max Line Length in terms of Number of Ngrams')
ax.set_ylabel('Percentage of songs that have a line with at most x ngrams')
plt.show()